# Specific Heat in Diamond and Lead

We assume to have a material with $3N$ phonon modes, of Einstein frequency $\omega_E$.

## A. System Partition Function

The partition function for such a system can be derived by:

So it is given by the expression:

$\text{Q} = \prod\limits^{3N}_{i = 1} \frac{1}{1 - \exp(-\beta\hbar\omega_E)}$

## B. Average Phonon Occupation Levels



## C. System Total Energy


## D. System Heat Capacity ($C_V$)




## E. Diamond Einstein Temperature ($T_E$) (by fit)

In this section we look at the datafile for measurements of the specific heat of diamond in order to obtain the Einstein temperature of the system by fitting. 

## F. Diamond Debye Temperature ($T_D$) (by fit)

We wish to find the Debye temperature of diamond, by fitting the Debye model to the measurements of specific heat. We use a formulation such that the solution is more stable for small values of temperature. 

## G. Debye Model vs. Einstein Model Comparison (for diamond)

## H. Debye Model vs. Einstein Model Comparison (for lead)

### H.a. Einstein Temperature ($T_E$) (by fit) - lead

### H.b. Debye Temperature ($T_D$) (by fit) - lead

### H.c. Model Comparison - lead

## I. Diamond vs. Lead Comparison Discussion

## J. Fermi Temperature ($T_F$) (by fit) - lead

## K. Room-Temperature Entropy by Debye Function - diamond & lead